In [ ]:
#pip install PyQt6

In [1]:
import json
import ollama
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import Ollama
import os

# base_path = os.path.dirname(os.path.abspath(__file__))
# file_path = os.path.join(base_path, "train_data.json")

# ✅ Įkelti įmonės duomenis iš JSON
with open('train_data.json', "r", encoding="utf-8") as f:
    company_data = json.load(f)

# ✅ Sukurti embeddings modelį
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# ✅ Paruošti FAISS duomenų bazę iš įmonės aprašymo ir paslaugų
texts = [f"{company_data['company']}: {company_data['description']}"]
contacts = [f"Puslapis: {company_data['website']}", f"El. paštas: {company_data['email']}", f"Telefonas: {company_data['phone']}"]
for service in company_data["services"]:
    service_text = f"{service['name']}: {service['description']} " \
                   f"Privalumai: {', '.join(service['benefits'])}. " \
                   f"Pavyzdys: {service['example']}"
    texts.append(service_text)

vector_store = FAISS.from_texts(texts, embeddings)
vector_store.save_local("faiss_index")

# ✅ Sukurti atmintį pokalbiui
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ✅ Pasirinkti efektyvų modelį su maža CPU apkrova
llm = Ollama(model="gemma2:2b-instruct-q5_1")

def chatbot(user_question):
    """Sugeneruoja atsakymą į klausimą, naudodamas FAISS ir pokalbio istoriją."""
    
    # 🔍 Gauti panašiausius dokumentus iš FAISS
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    retrieved_docs_with_scores = vector_store.similarity_search_with_score(user_question, k=3)

    # 📌 Filtruoti tik geriausius rezultatus
    threshold = 0.7
    filtered_docs = [doc[0].page_content for doc in retrieved_docs_with_scores if doc[1] > threshold]

    # 📝 Gauti pokalbio istoriją
    chat_history = memory.load_memory_variables({}).get("chat_history", [])

    # 🔹 Sudaryti promptą su kontekstu
    prompt = (f"Tu esi AI asistentas, kuris atsakinėja apie įmonę '{company_data['company']}'. \n"
              f"Įmonės kontaktinė informacija:\n"
              f"{' '.join(contacts)}\n\n"
              f"Remkis tik žemiau pateikta informacija ir venk spėliojimo.\n\n"
              f"Pokalbio istorija: {chat_history}\n"
              f"Klausimas: {user_question}\n"
              f"Kontekstas: {filtered_docs if filtered_docs else 'Nėra atitinkamos informacijos'}\n"
              f"Atsakymas:")

    # 🧠 Generuoti atsakymą
    response = llm.invoke(prompt)

    # 🔄 Išsaugoti pokalbio istoriją
    memory.save_context({"input": user_question}, {"output": response})

    return response

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
print(chatbot("Testas"))

MD Baltics yra verslo IT paslaugų teikėjas, specializuojasi duomenų integruojimo, sąsajų kūrimo, verslo valdymo sistemų parengimu ir diegimu bei IT konsultingai. 

**Pagrindiniai MD Baltics paslaugų tipai:**

* **Duomenų Integravimas ir Sąsajų Kūrimas**: Integruoja skirtingas IT sistemas, kad duomenys būtų valdomi sklandžiai, prieinami realiu laiku ir automatiškai atnaujinami be rankinio įsikišimo.
* **Verslo Valdymo Sistemos**: Parenkame ir diegiame HR, CRM, ERP sprendimus pagal įmonės poreikius.
* **API Integracijos**: Integruojame ERP, CRM, sandėlio valdymo, apskaitos ir kitas verslo valdymo sistemas. 


**MD Baltics privalomai:**

* Pašalinamas rankinio įvedimo poreikis
* Optimizuojamas duomenų srautas
* Užtikrinamas didesnis tikslumas
* Didesnis darbuotojų efektyvumas
* Automatizuotas darbo krūvio paskirstymas


**Pavyzdžiai:**

* Automatinė sandėlio ir apskaitos sistemų integracija sumažino klaidų kiekį 80% ir išvengė 50 000 € nuostolių.
* Centralizuota verslo valdymo sistema sum

In [ ]:
# https://dash.plotly.com/ 